This notebook will read data from the InfluxDB and use it to train the TCN model. Then it will use this model to continuously output an anomaly score back to the InfluxDB which can be output there via bucket-->anomalies-->tcn-->anomaly_score

In [1]:
###Enter the information from the InfluxDB here.
###In particular, be sure not to remove the outer quotation marks for the bucket and the measurement variables.
###Fields need to be given as lists

organisation = "manubrain"
bucket = '"sinus"'
msm = '"anomalie_detection"'
fields = ['field_0','field_1','field_2','field_3','field_4','field_5','field_6','field_7','field_8','field_9']

###Give the time_window for how far we should go into the past to construct the training data
time_window = "1h"

###Give the desired number of training epochs of the method
epochs = 10


In [ ]:
!su jovyan

]0;jovyan@jupyter: ~/workjovyan@jupyter:~/work$ 

In [1]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install -e /home/jovyan/detect

Obtaining file:///home/jovyan/detect
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/890.1 MB 4.5 MB/s eta 0:02:54^C
     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/890.1 MB 4.5 MB/s eta 0:02:53
ERROR: Operation cancelled by user


In [4]:
!ls /home/jovyan/detect


ls: cannot open directory '/home/jovyan/detect': Permission denied


In [3]:
from mb_detect.models.classic import sklearn_dect
from mb_detect.models.deep.tcn_model import TCN

In [4]:
import pandas as pd
import torch

In [5]:
# CONFIGURATION
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [6]:
url = "http://influxdb:8086"
token = os.environ.get("INFLUXDB_TOKEN")
client = influxdb_client.InfluxDBClient(url=url, token=token, org=organisation)
query_api = client.query_api()

The fields need to be brought in the correct form for the query api

In [7]:
field_query = ''

if len(fields) == 1:
    field_query = 'r["_field"] == ' + str(fields[0])
    
else:

    for i in range(len(fields)-1):
        field_query = field_query + 'r["_field"] == "' + fields[i] + '"' + ' or '
        
    field_query = field_query + 'r["_field"] == "' + fields[-1] + '"'

In [8]:
# Query script for loading the data from the influxDB

def data_loader(bucket,msm,field_query,time_window="10s"):

    query = 'from(bucket:{})\
    |> range(start: -{})\
    |> filter(fn:(r) => r._measurement == {})\
    |> filter(fn:(r) => {})\
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")\
    '.format(bucket,time_window,msm,field_query)

    data = query_api.query_data_frame(org="manubrain", query=query)
    
    ###We need to rename to columns to the name 'value_nr' in order to be handable for the method
    rename = {}
    [rename.update({c:c.replace("field", "value")}) for c in data.columns if c.startswith("field")]
    data.rename(columns=rename, inplace=True)
    
    return(data)

In [9]:
train_data = data_loader(bucket,msm,field_query,time_window=time_window)

Train the TCN model on the chosen train data with the chosen number of epochs. 

In [10]:
X = train_data[[c for c in train_data.columns if c.startswith("value")]]
model = TCN(X.shape[1])
model.fit(X,epochs=epochs) #turn up epochs for better learning

The model is used to make predictions on the next data point based on the last 5 data points

In [11]:
def make_predictions_tcn(model):
    import numpy as np
    test_data = data_loader(bucket,msm,field_query,time_window="10s")
    test_data = test_data[[c for c in train_data.columns if c.startswith("value")]]
    tx = torch.Tensor(test_data.to_numpy()).T
    history = tx[...,-6:-1]
    predict_target = tx[...,-1].numpy()
    predict_model = model.forward(history).detach().numpy().flatten()
    predict_error = np.linalg.norm(predict_target-predict_model)
    return [predict_error,predict_target,predict_model]

Write data

In [12]:
write_api = client.write_api(write_options=SYNCHRONOUS)
while True:
    predictions = make_predictions_tcn(model)
    predict_error = predictions[0]

    point = (
      Point("anomalies")
      .tag("scores", 'anomaly_score')
      .field("tcn", predict_error)
      )
    write_api.write(bucket=bucket.replace('"',""), org=organisation, record=point)
    time.sleep(1)

KeyboardInterrupt: 